# Question Answering using Embeddings

Many use cases require GPT-3.5 Turbo to respond to user questions with insightful answers. For example, a customer support chatbot may need to provide answers to common questions. The GPT models have picked up a lot of general knowledge in training, but we often need to ingest and use a large library of more specific information.

In this notebook we will demonstrate a method for enabling GPT-3.5 Turbo to answer questions using a library of text as a reference, by using document embeddings and retrieval. We'll be using a dataset of Wikipedia articles about the 2024 Summer Olympic Games. Please see [this notebook](fine-tuned_qa/olympics-1-collect-data.ipynb) to follow the data gathering process.

In [2]:
import numpy as np
import pandas as pd
import openai
import pickle
import tiktoken
from transformers import GPT2TokenizerFast
import dotenv

# Configure your OpenAI API key and select the appropriate models
dotenv.load_dotenv()
client = openai.OpenAI()

# Model for text generation
COMPLETIONS_MODEL = "gpt-3.5-turbo"

# Model for generating embeddings
EMBEDDING_MODEL = "text-embedding-3-small"

By default, GPT-3 isn't an expert on the 2024 Olympics since the knowledge cutoff date is September 2021:

In [10]:
prompt = "Who won the 2024 Summer Olympics men's high jump?"

client.chat.completions.create(
    model=COMPLETIONS_MODEL,
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
    max_tokens=300,
).choices[0].message.content.strip(" \n")

"I'm sorry, but I cannot provide real-time information as I am an AI assistant and do not have access to current data. Please check the official Olympics website or news sources for the most up-to-date information on the winner of the men's high jump at the 2024 Summer Olympics."

Evidently GPT-3 needs some assistance here. 

The first issue to tackle is that the model is hallucinating an answer rather than telling us "I don't know". This is bad because it makes it hard to trust the answer that the model gives us! 

# 0) Preventing hallucination with prompt engineering

We can address this hallucination issue by being more explicit with our prompt:


In [12]:
prompt = """Answer the question as truthfully as possible, and if you're unsure of the answer, say "Sorry, I don't know".

Q: Who won the 2024 Summer Olympics men's high jump?
A:"""

client.chat.completions.create(
    model=COMPLETIONS_MODEL,
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
    max_tokens=300,
).choices[0].message.content.strip(" \n")

"Sorry, I don't know."

To help the model answer the question, we provide extra contextual information in the prompt. When the total required context is short, we can include it in the prompt directly. For example we can use this information taken from Wikipedia. We update the initial prompt to tell the model to explicitly make use of the provided text.

In [14]:
prompt = """Answer the question as truthfully as possible using the provided text, and if the answer is not contained within the text below, say "I don't know"

Context:
Athletics (track and field) rulebooks all across the world provide for the same procedure to break ties for first place in a vertical jump. 
They do not have a means of enforcement; you can't make the tied jumpers jump. Jump offs were held at major championships for over a hundred years until the 
previous Olympics when both Mutaz Essa Barshim and Gianmarco Tamberi agreed to share the gold medal at Barshim's suggestion. 
Since then Nina Kennedy and Katie Moon also agreed to share the gold medal in the Women's Pole Vault at the 2023 World Championships. 
It has been a subject of discussion. Both Barshim and Tamberi return, Tamberi as seasonal world leader. #2 Hamish Kerr has been outspoken online that he will 
not be sharing a gold if it comes to that. Barshim won the 2022 World Championships ahead of Woo Sang-hyeok and Andriy Protsenko. Tamberi won in 2023 over 
JuVaughn Harrison and Barshim.

During the 2024 Summer Olympics men's high jump qualifying round, Barshim struggled with severe cramps. His friend in gold, Tamberi, rushed over to help 
massage the cramping calf. Protsenko couldn't get over a bar. Harrison topped out at 2.20 and didn't advance. It was so tight, two people who cleared 
2.24m but had excessive misses did not advance.

Hours before the final, Tamberi was taken to the Emergency Room, vomiting blood. Heroically, he made it to the stadium and even managed to clear 2.27m but 
was not able to go higher. Six jumpers were able to get over 2.31m, Shelby McEwen and Barshim still had perfect rounds going. Kerr took three attempts to get 
over. At 2.34m, Stefano Sottile, Kerr and Barshim got over on their first attempts, putting Barshim in first place still with a perfect round going. McEwen 
jumped over the bar cleanly for a new personal best clearing it on his third attempt. Moving the bar to 2.36m, after Barshim and Sottile missed, McEwen flew 
over the bar on his first attempt, celebrating his second personal best of the competition in the pit. Moments later, Kerr also cleared it cleanly on his 
first attempt. No matter what Barshim and Sottile did at this height, McEwen and Kerr were tied with a first attempt clearance of the most recent height and 
two total misses in the competition. Barshim took one more attempt then passed for one remaining hero jump at the next height. Sottile took both of his remaining 
attempts and after missing the second was guaranteed fourth place. At 2.38m, Barshim missed his attempt leaving him with the bronze medal. McEwen and Kerr both 
missed all three of their attempts at what would be their personal bests. But they were still tied.

True to his online statement, Kerr wanted to keep going. There was going to be a jump off. The jumping order remained the same, McEwen jumping first and Kerr 
jumping last. The first step was to jump at the height they had just missed. Both missed again. So next they are to jump at the height they last made. Again 
both missed. So next they are to jump one height back, 2.34m. Both athletes were now on their 14th attempt of the competition. McEwen missed, then Kerr cleared. 
He leaped out of the pit and ran into the infield to celebrate his victory.

Q: Who won the 2024 Summer Olympics men's high jump?
A:"""

client.chat.completions.create(
    model=COMPLETIONS_MODEL,
    messages=[{"role": "user", "content": prompt}],
    temperature=0,
    max_tokens=300,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
).choices[0].message.content.strip(" \n")

'Hamish Kerr'

Adding extra information into the prompt only works when the dataset of extra content that the model may need to know is small enough to fit in a single prompt. What do we do when we need the model to choose relevant contextual information from within a large body of information?

**In the remainder of this notebook, we will demonstrate a method for augmenting GPT-3 with a large body of additional contextual information by using document embeddings and retrieval.** This method answers queries in two steps: first it retrieves the information relevant to the query, then it writes an answer tailored to the question based on the retrieved information. The first step uses the [Embeddings API](https://beta.openai.com/docs/guides/embeddings), the second step uses the [Completions API](https://beta.openai.com/docs/guides/completion/introduction).
 
The steps are:
* Preprocess the contextual information by splitting it into chunks and create an embedding vector for each chunk.
* On receiving a query, embed the query in the same vector space as the context chunks and find the context embeddings which are most similar to the query.
* Prepend the most relevant context embeddings to the query prompt.
* Submit the question along with the most relevant context to GPT, and receive an answer which makes use of the provided contextual information.

# 1) Preprocess the document library

We plan to use document embeddings to fetch the most relevant part of parts of our document library and insert them into the prompt that we provide to GPT-3. We therefore need to break up the document library into "sections" of context, which can be searched and retrieved separately. 

Sections should be large enough to contain enough information to answer a question; but small enough to fit one or several into the GPT-3 prompt. We find that approximately a paragraph of text is usually a good length, but you should experiment for your particular use case. In this example, Wikipedia articles are already grouped into semantically related headers, so we will use these to define our sections. This preprocessing has already been done in [this notebook](fine-tuned_qa/olympics-1-collect-data.ipynb), so we will load the results and use them.

In [3]:
# We have hosted the processed dataset, so you can download it directly without having to recreate it.
# This dataset has already been split into sections, one row for each section of the Wikipedia page.

df = pd.read_csv('../data/Series4000.csv', header=0)
# df = df.set_index(["title", "heading"])
# df = df.set_axis(['title', 'heading'] + list(df.columns[2:]), axis=1)
print(f"{len(df)} rows in the data.")
df.sample(5)


9 rows in the data.


,title,heading1,heading2,heading3,heading4,content
0,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.1 The Mortgage application,"(a) Required use of Form 65, Uniform Residenti...","Form 65, Uniform Residential Loan Application,..."
5,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(c) Mortgage instruments for ARMs - Required A...,Required ARM Uniform Instruments\nARMs must be...
3,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(a) Use of Uniform Instruments,The Security Instrument and Note must be execu...
2,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.1 The Mortgage application,(c) Electronic and fax copies of loan applicat...,Freddie Mac agrees that the Seller may receive...
7,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(c) Mortgage instruments for ARMs - Instructio...,"The Seller must complete Section 4(D), Limits ..."


In [4]:
# TODO:
# Create a new column called "context" that combines multiple fields into a single text string.
#
# Think about:
# - Which columns should be included (e.g., title, headings, content)?
# - How you want to separate different fields (commas, semicolons, new lines, labels, etc.).
# - Whether you need to clean the text (e.g., strip extra whitespace).
#
# Try at least one of the following strategies:
# 1. String concatenation using "+" operators
# 2. Using Python f-strings
# 3. Applying a function row-wise with df.apply(...)
#
# Goal:
# Produce a single text field that can later be used for tasks like search, embeddings, or retrieval.
for index, row in df.iterrows():
    df.at[index, 'context'] = f"Title: {str(row['title']).strip()}\n" + \
        f"Heading1: {str(row['heading1']).strip()}\n" + \
        f"Heading2: {str(row['heading2']).strip()}\n" + \
        f"Heading3: {str(row['heading3']).strip()}\n" + \
        f"Heading4: {str(row['heading4']).strip()}\n" + \
        f"Content: {str(row['content']).strip()}"
df.sample(5)

,title,heading1,heading2,heading3,heading4,content,context
4,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(b) Additional Mortgage documentation requirem...,In addition to the Uniform Instruments require...,Title: Series 4000: Mortgage Eligibility\nHead...
3,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(a) Use of Uniform Instruments,The Security Instrument and Note must be execu...,Title: Series 4000: Mortgage Eligibility\nHead...
2,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.1 The Mortgage application,(c) Electronic and fax copies of loan applicat...,Freddie Mac agrees that the Seller may receive...,Title: Series 4000: Mortgage Eligibility\nHead...
6,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(c) Mortgage instruments for ARMs - Use of Fan...,The Seller may use Fannie Mae's ARM instrument...,Title: Series 4000: Mortgage Eligibility\nHead...
5,Series 4000: Mortgage Eligibility,Topic 4100: Uniform Instruments,Chapter 4101: Uniform Instruments,4101.2 Home Mortgage Uniform Instruments,(c) Mortgage instruments for ARMs - Required A...,Required ARM Uniform Instruments\nARMs must be...,Title: Series 4000: Mortgage Eligibility\nHead...


## 1.1 Create some questions based on context

In [5]:

def get_questions(context):
    try:
        response = client.chat.completions.create(
            model=COMPLETIONS_MODEL,
            messages=[{"role": "user", "content": f"Write questions based on the text below\n\nText: {context}\n\nQuestions:\n1."}],
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=["\n\n"]
        )
        return response.choices[0].message.content
    except:
        return ""


df['questions']= df.context.apply(get_questions)
df['questions'] = "1. " + df.questions
print(df[['questions']].values[0][0])

1. What form must be used for all Mortgage applications?
2. Where can the Seller find the current version of Form 65 and the UMDP Instructions?
3. Can the Seller make changes to the style and formatting of Form 65?
4. What are the components of Form 65?
5. Are translation aids available for Form 65 and its components?


## 1.2 Create answers based on the context

In [6]:
def get_answers(row):
    try:
        response = client.chat.completions.create(
            model=COMPLETIONS_MODEL,
            messages=[{"role": "user", "content": f"Write answer based on the text below\n\nText: {row.context}\n\nQuestions:\n{row.questions}\n\nAnswers:\n1."}],
            temperature=0,
            max_tokens=257,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].message.content
    except Exception as e:
        print (e)
        return ""


df['answers']= df.apply(get_answers, axis=1)
df['answers'] = "1. " + df.answers
df = df.dropna().reset_index().drop('index',axis=1)
print(df[['answers']].values[0][0])

1. Form 65, Uniform Residential Loan Application, must be used for all Mortgage applications.
2. The Seller can find the current version of Form 65 and the UMDP Instructions in Exhibit 4A, Single-Family Uniform Instruments.
3. The Seller may make changes to the style and formatting of Form 65 and its components in accordance with the UMDP Rendering Options.
4. The components of Form 65 are Borrower Information, Additional Borrower, Continuation Sheet, Lender Loan Information, and the Unmarried Addendum.
5. Translation aids are available for Form 65 and its components on Freddie Mac’s Multi-language Resources for Lenders and Other Housing Professionals web page.


In [7]:
def num_tokens(text: str, model: str = "gpt-4") -> int:
    """Return the number of tokens in a string."""
    encoding = tiktoken.encoding_for_model(model)
    return len(encoding.encode(text))

df['tokens'] = df.apply(lambda row : num_tokens(row['context']), axis = 1)

In [8]:
df.to_csv('../data/selling_qa.csv', index=False)

We preprocess the document sections by creating an embedding vector for each section. An embedding is a vector of numbers that helps us understand how semantically similar or different the texts are. The closer two embeddings are to each other, the more similar are their contents. See the [documentation on OpenAI embeddings](https://beta.openai.com/docs/guides/embeddings) for more information.

This indexing stage can be executed offline and only runs once to precompute the indexes for the dataset so that each piece of content can be retrieved later. Since this is a small example, we will store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/) or [Weaviate](https://github.com/semi-technologies/weaviate) to power the search.

In [9]:
# TODO:
# Implement a function that converts a piece of text into an embedding vector
# using the OpenAI Embeddings API.
#
# Think about:
# - Which embedding model to use
# - What parameters the API expects
# - How to extract the embedding from the API response
#
# Function signature should remain the same.
def get_embedding(text: str, model: str = EMBEDDING_MODEL):
    resp = client.embeddings.create(model=model, input=text)
    return resp.data[0].embedding


# TODO:
# Implement a function that creates embeddings for each row in a DataFrame.
#
# Requirements:
# - Generate one embedding per document/row
# - Associate each embedding with the row index
#
# Think about:
# - Iterating with df.iterrows(), df.itertuples(), or vectorized approaches
# - Whether batching API calls would be beneficial
# - How this approach would scale for large datasets
#
# Return:
# A dictionary mapping row index -> embedding vector
def compute_doc_embeddings(df: pd.DataFrame):
    """
    Create an embedding for each row in the dataframe using the OpenAI Embeddings API.
    """
    embeddings = {}
    for idx, row in df.iterrows():
        embeddings[idx] = get_embedding(str(row.get('context', '')))
    return embeddings

In [10]:
document_embeddings = compute_doc_embeddings(df)

In [11]:
# JSON save and load
import json

def jsonKeys2int(x):
    if isinstance(x, dict):
        return {int(k):v for k,v in x.items()}
    return x

with open('../data/embeddings.json', 'w') as fp:
    json.dump(document_embeddings, fp)
    
with open('../data/embeddings.json', 'r') as fp:
    document_embeddings = json.load(fp, object_hook=jsonKeys2int)

In [12]:
def load_embeddings(fname: str):
    """
    Read the document embeddings and their keys from a CSV.
    
    fname is the path to a CSV with exactly these named columns: 
        "title", "heading", "0", "1", ... up to the length of the embedding vectors.
    """
    
    df = pd.read_csv(fname, header=0)
    max_dim = max([int(c) for c in df.columns if c != "title" and c != "heading"])
    return {
           (r.title, r.heading): [r[str(i)] for i in range(max_dim + 1)] for _, r in df.iterrows()
    }

Again, we have hosted the embeddings for you so you don't have to re-calculate them from scratch.

In [15]:
# document_embeddings = load_embeddings("https://cdn.openai.com/API/examples/data/olympics_sections_document_embeddings.csv")

# ===== OR, uncomment the below line to recaculate the embeddings from scratch. ========

# document_embeddings = compute_doc_embeddings(df)

In [13]:
# An example embedding:
example_entry = list(document_embeddings.items())[0]
print(f"{example_entry[0]} : {example_entry[1][:5]}... ({len(example_entry[1])} entries)")

0 : [0.007922090590000153, 0.037442781031131744, 0.045801009982824326, 0.04306701198220253, -0.020843496546149254]... (1536 entries)


So we have split our document library into sections, and encoded them by creating embedding vectors that represent each chunk. Next we will use these embeddings to answer our users' questions.

# 2) Find the most similar document embeddings to the question embedding

At the time of question-answering, to answer the user's query we compute the query embedding of the question and use it to find the most similar document sections. Since this is a small example, we store and search the embeddings locally. If you have a larger dataset, consider using a vector search engine like [Pinecone](https://www.pinecone.io/) or [Weaviate](https://github.com/semi-technologies/weaviate) to power the search.

In [14]:
def vector_similarity(x, y):
    """
    Returns the similarity between two vectors.
    
    Because OpenAI Embeddings are normalized to length 1, the cosine similarity is the same as the dot product.
    """
    return np.dot(np.array(x), np.array(y))

def order_document_sections_by_query_similarity(query, contexts):
    """
    Find the query embedding for the supplied query, and compare it against all of the pre-calculated document embeddings
    to find the most relevant sections. 
    
    Return the list of document sections, sorted by relevance in descending order.
    """
    query_embedding = get_embedding(query)
    
    document_similarities = sorted([
        (vector_similarity(query_embedding, doc_embedding), doc_index) for doc_index, doc_embedding in contexts.items()
    ], reverse=True)
    
    return document_similarities

In [15]:
order_document_sections_by_query_similarity("Can I use premium financing to fund the down payment?", document_embeddings)[:5]

[(np.float64(0.3598545738780053), 3),
 (np.float64(0.3498147755531129), 4),
 (np.float64(0.34772783858981904), 7),
 (np.float64(0.3472669527563551), 6),
 (np.float64(0.3455815209188211), 2)]

In [16]:
order_document_sections_by_query_similarity("Can I use premium financing to fund closing costs and prepaids?", document_embeddings)[:5]

[(np.float64(0.3788272456786729), 2),
 (np.float64(0.3723193874925157), 4),
 (np.float64(0.3704394695787016), 7),
 (np.float64(0.36882649050376737), 1),
 (np.float64(0.3660716986970437), 6)]

We can see that the most relevant document sections for each question include the summaries for the Men's and Women's high jump competitions - which is exactly what we would expect.

# 3) Add the most relevant document sections to the query prompt

Once we've calculated the most relevant pieces of context, we construct a prompt by simply prepending them to the supplied query. It is helpful to use a query separator to help the model distinguish between separate pieces of text.

In [20]:
MAX_SECTION_LEN = 1000
SEPARATOR = "\n* "

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
separator_len = len(tokenizer.tokenize(SEPARATOR))

f"Context separator contains {separator_len} tokens"

'Context separator contains 3 tokens'

In [21]:
def construct_prompt(question: str, context_embeddings: dict, df: pd.DataFrame) -> str:
    """
    Fetch relevant 
    """
    most_relevant_document_sections = order_document_sections_by_query_similarity(question, context_embeddings)
    
    chosen_sections = []
    chosen_sections_len = 0
    chosen_sections_indexes = []
     
    for _, section_index in most_relevant_document_sections:
        # Add contexts until we run out of space.        
        document_section = df.loc[section_index]
        
        chosen_sections_len += document_section.tokens + separator_len
        if chosen_sections_len > MAX_SECTION_LEN:
            break
            
        chosen_sections.append(SEPARATOR + document_section.content.replace("\n", " "))
        chosen_sections_indexes.append(str(section_index))
            
    # Useful diagnostic information
    print(f"Selected {len(chosen_sections)} document sections:")
    print("\n".join(chosen_sections_indexes))
    
    header = """Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."\n\nContext:\n"""
    
    return header + "".join(chosen_sections) + "\n\n Q: " + question + "\n A:"

In [22]:
prompt = construct_prompt(
    "What is the required use of Form 65?",
    document_embeddings,
    df
)

print("===\n", prompt)

Selected 2 document sections:
0
1
===
 Answer the question as truthfully as possible using the provided context, and if the answer is not contained within the text below, say "I don't know."

Context:

* Form 65, Uniform Residential Loan Application, must be used for all Mortgage applications.  The Seller must use the version of Form 65 that is current as of the date of the loan application. See Exhibit 4A, Single-Family Uniform Instruments, for the date of the current version of Form 65 and the most current version of the Uniform Mortgage Data Program® (UMDP®) Instructions for Completing the Uniform Residential Loan Application.  Seller may make changes to the style and formatting of the Form 65 and its components – Borrower Information, Additional Borrower, Continuation Sheet, Lender Loan Information and the Unmarried Addendum, if applicable, in accordance with the UMDP Rendering Options for the Uniform Residential Loan Application, Document revised 1/2020(PDF 5mb opens in new window

We have now obtained the document sections that are most relevant to the question. As a final step, let's put it all together to get an answer to the question.

# 4) Answer the user's question based on the context.

Now that we've retrieved the relevant context and constructed our prompt, we can finally use the Completions API to answer the user's query.

In [23]:
COMPLETIONS_API_PARAMS = {
    # We use temperature of 0.0 because it gives the most predictable, factual answer.
    "temperature": 0.0,
    "max_tokens": 300,
    "model": COMPLETIONS_MODEL
}

In [24]:
def answer_query_with_context(
    query: str,
    df: pd.DataFrame,
    document_embedding,
    show_prompt: bool = False
) -> str:
    prompt = construct_prompt(
        query,
        document_embeddings,
        df
    )
    
    if show_prompt:
        print(prompt)

    response = client.chat.completions.create(
                messages=[{"role": "user", "content": prompt}],
                **COMPLETIONS_API_PARAMS
            )

    return response.choices[0].message.content.strip(" \n")

In [25]:
answer_query_with_context("What is the required use of Form 65?", df, document_embeddings)

Selected 2 document sections:
0
1


'Form 65, Uniform Residential Loan Application, must be used for all Mortgage applications.'

In [26]:
answer_query_with_context("What are the Seller's formatting options for Form 65?", df, document_embeddings)

Selected 2 document sections:
0
2


'The Seller may make changes to the style and formatting of Form 65 and its components in accordance with the UMDP Rendering Options for the Uniform Residential Loan Application. The fields names, descriptions, and order of sections may not be altered in any way, but form fields within a section may be moved within that section if additional field length is needed. Any adjustments made to the format of the form must be made pursuant to all applicable law.'

In [27]:
answer_query_with_context("What are the translation aids for Form 65?", df, document_embeddings)

Selected 2 document sections:
0
1


'Translation aids for Form 65 and its components are available on Freddie Mac’s Multi-language Resources for Lenders and Other Housing Professionals web page.'

## Test with more examples

In [28]:
query = "The Seller has formatting options for Form 65, which must be in accordance with the UMDP Rendering Options."
answer = answer_query_with_context(query, df, document_embeddings)

print(f"\nQ: {query}\nA: {answer}")

Selected 2 document sections:
0
2

Q: The Seller has formatting options for Form 65, which must be in accordance with the UMDP Rendering Options.
A: True
